In [51]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from sklearn.metrics._ranking import roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [52]:
df= pd.read_csv('data/train_final.csv')
X=df.iloc[:,2:]
Y=df.iloc[:,1]
X.head()


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,25884,1,33.63,118596,1,0,118595,125738,1,3,...,1945,118450,119184,1,121372,1,1,1,2,1
1,34346,1,10.62,118041,1,0,117902,130913,1,1,...,15385,117945,292795,1,259173,1,1,1,1,1
2,34923,1,1.77,118327,1,0,117961,124402,1,2,...,7547,118933,290919,1,118784,1,1,1,1,1
3,80926,1,30.09,118300,1,0,117961,301218,1,0,...,4933,118458,118331,1,307024,1,1,1,2,1
4,4674,1,1.77,119921,1,0,119920,302830,1,0,...,13836,142145,4673,1,128230,1,1,1,620,1


In [53]:
df.shape

for i in df.columns:
    print(len(df[i].unique()))

16383
2
5170
7
168
162
8
9
118
1851
7
182
7
157
322
11349
3555
432
67
9
322
7
8
7
906
7


In [54]:
print(df['f5'].unique())

[1 2 3 5 6 7 4 9]


In [55]:
# split data into train and test sets
seed = 39
X_train, X_rem, y_train, y_rem = train_test_split(X,Y,test_size=0.30,random_state=seed)
X_test, X_val, y_test, y_val = train_test_split(X_rem, y_rem, test_size =0.5,random_state=seed)
print(X_train)

          f1  f2    f3      f4  f5  f6      f7      f8  f9  f10  ...     f15  \
781    31833   1  1.77  117883   1   0  117882  123932   1    4  ...   17147   
14482  13877   1  1.77   31010   1   0  118185  147516   1    2  ...    1999   
7080   43713   1  1.77  118386   1   0  117961  118785   1    1  ...   18686   
15536  43169   1  1.77  118386   1   0  117961  117906   1    1  ...   71171   
8108   35539   1  1.77  117940   1   0  117929  117886   1    0  ...   12264   
...      ...  ..   ...     ...  ..  ..     ...     ...  ..  ...  ...     ...   
7060   43535   1  1.77  118327   1   0  117961  149217   1    0  ...    2610   
1088   25992   1  3.54  118225   1   0  117961  127477   1    6  ...   15824   
7209   16088   1  1.77  118225   1   0  117961  117906   1    0  ...   17790   
8297   31440   1  7.08  118413   1   0  117961  127475   1    0  ...    2296   
11657  24013   1  1.77  118300   1   0  117961  240983   1    0  ...  111936   

          f16     f17  f18     f19  f20

In [56]:
# fit model no training data
model=XGBClassifier(n_estimators=5000,  n_jobs=-1, seed=seed)
model.fit(X_train, y_train, verbose=100, early_stopping_rounds=100, eval_set=[(X_train,y_train),(X_val,y_val)])

[22:48:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.47541	validation_1-logloss:0.47604
[100]	validation_0-logloss:0.03331	validation_1-logloss:0.13503
[189]	validation_0-logloss:0.01249	validation_1-logloss:0.15036


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=5000, n_jobs=-1, num_parallel_tree=1,
              random_state=39, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=39, subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [57]:
test_df=pd.read_csv('data/test_final.csv')
X_test_output=test_df.drop(labels='Id',axis=1)
y_pred_output=model.predict_proba(X=X_test_output)[:,1]
y_pred=model.predict_proba(X=X_test)[:,1]
print(len(y_pred))
y_pred=np.array(y_pred)
Id=pd.read_csv('data/test_final.csv')
Id=Id.iloc[:,0].values
print(len(Id))

2457
16385


In [58]:
output_df = pd.DataFrame({"Id" : Id, "Y" : y_pred_output})
output_df.to_csv("y_pred.csv", index=False)
score = roc_auc_score(y_test, y_pred)
print(score)

0.8696275634477882


17, 39 with highest random seeds

